In [1]:
import random
from importlib import reload

import torch
from torch.nn.functional import softmax, log_softmax

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM

from util import mt

In [255]:
model_name = 'EleutherAI/gpt-neo-125M'
model_name = 'facebook/xglm-7.5B'
model_name = 'facebook/xglm-4.5B'
model_name = 'facebook/xglm-2.9B'

In [256]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).eval().half().cuda()

In [398]:
src = load_dataset("gsarti/flores_101", 'zho_simpl')
tgt = load_dataset("gsarti/flores_101", 'eng')
eng = load_dataset("gsarti/flores_101", 'eng')

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/zho_simpl/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

In [399]:
src_lang = 'Chinese'
tgt_lang = 'English'

In [7]:
def tokenize(s):
  ids = []
  i = 0
  t = 0
  while t < len(s):
    c = s[t]
    if c == '\n':
      ids += s_ids(s[i:t]) + [2]
      i = t+1
      t = t+1
    else:
      t += 1
  ids += s_ids(s[i:])
  return ids

In [8]:
def s_ids(s):
  return tokenizer(s).input_ids[1:]

In [9]:
def make_demo_prompt(src_endonym, tgt_endonym, src, tgt, k):
  src_points = random.sample(list(src), k=k)
  tgt_points = [ tgt[point['id']-1] for point in src_points ]
  demos = []
  for s, t in zip(src_points, tgt_points):
    demo = f"{src_endonym} : {s['sentence']} {tgt_endonym} : {t['sentence']}"
    demos.append(demo)
  demo_prompt_ids = [2]
  for demo in demos:
    demo_prompt_ids.extend(s_ids(demo))
    demo_prompt_ids.append(2)
  return demo_prompt_ids

In [1443]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [400]:
src_points = random.sample(list(src['dev']), k=10)
tgt_points = [ tgt['dev'][point['id']-1] for point in src_points ]

In [401]:
demo_prompt_ids = mt.make_demo_prompt(src_points, tgt_points, src_lang, tgt_lang, tokenizer)
uncond_demo_prompt_ids = mt.make_uncond_demo_prompt(tgt_points, tokenizer)

In [12]:
inference_point = src['dev'][idx] 
cond_prompt_ids = uncond_demo_prompt_ids + s_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :")
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {tgt_lang} :")
uncond_prompt_t = torch.LongTensor([[2]]).to(model.device)
print(idx)
print()
print(eng['dev'][idx]['sentence'])
print()
print(src['dev'][idx]['sentence'])
print()
print(tgt['dev'][idx]['sentence'])

NameError: name 'idx' is not defined

In [392]:
inference_point = src['dev'][idx] 
cond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :")
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {tgt_lang} :")
uncond_prompt_t = torch.LongTensor([uncond_demo_prompt_ids]).to(model.device)
print(eng['dev'][idx]['sentence'])
print()
print(src['dev'][idx]['sentence'])
print()
print(tgt['dev'][idx]['sentence'])

When the tenants started sharing what had occurred to them, most of the families involved suddenly realized that Carolyn Wilson of the OHA had stolen their security deposits, and skipped out of town.

Kun vuokralaiset alkoivat kertoa, mitä heille oli tapahtunut, useimmat perheet, joita asia koski, käsittivät pian, että Oaklandin asuntoviraston edustaja Carolyn Wilson varastanut heidän takuutalletuksensa ja paennut kaupungista.

When the tenants started sharing what had occurred to them, most of the families involved suddenly realized that Carolyn Wilson of the OHA had stolen their security deposits, and skipped out of town.


In [501]:
idx = 200

In [621]:
idx += 1

In [622]:
inference_point = src['dev'][idx] 
cond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :")
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + s_ids(f"{tgt_lang} :")
uncond_prompt_t = torch.LongTensor([uncond_prompt_ids]).to(model.device)
print(eng['dev'][idx]['sentence'])
print()
print(src['dev'][idx]['sentence'])
print() 
print(tgt['dev'][idx]['sentence'])

The man allegedly drove a three-wheeled vehicle armed with explosives into a crowd.

据说该男子驾驶一辆装有炸药的三轮车驶进了人群当中。

The man allegedly drove a three-wheeled vehicle armed with explosives into a crowd.


In [623]:
with torch.no_grad():
  gen_ids = mt.pg_greedy(model, 5, 1, cond_prompt_t, uncond_prompt_t, 50, tokenizer=tokenizer, p=0.6)
print()
print(tokenizer.decode(gen_ids[0].tolist()))


According to some reports, the man driving a three-wheeled cart entered the crowd.</s> Chinese : 该男子驾驶一辆装有炸药的三轮车驶进了人群当中。 English : According to some reports, the man driving a


In [624]:
with torch.no_grad():
  greedy_gen_ids = mt.greedy(model, cond_prompt_t, 50)
print(tokenizer.decode(greedy_gen_ids[0].tolist()))

According to some reports, the man was driving a three-wheeled cart with explosives.</s> Chinese : 该公司在其网站上表示,该公司将于今年晚些时候在伦敦和巴黎开设新的办公室。


In [500]:
idx

91

In [1125]:
tokenizer.decode(cond_prompt_t[0])

'</s> Residents from other areas often cite family-friendliness as a primary reason for moving there, and visitors often find the city easy to enjoy with children around.</s> Enceladus is the most reflective object in the solar system, reflecting about 90 percent of the sunlight that hits it.</s> Beaches, theme parks and camp grounds are often the most common places frequented by recreational tourists.</s> He originally named the Hangeul alphabet Hunmin Jeongeum, which means "the correct sounds for the instruction of the people".</s> They need to show the insurer\'s e-mail address and international phone numbers for advice/authorizations and making claims.</s> The Drukgyal Dzong is a ruined fortress and Buddhist monastery in the upper part of the Paro District (in Phondey Village).</s> Last week, Naked News announced that it would dramatically increase its international language mandate to news reporting, with three new broadcasts.</s> Animals are found all over the earth. They dig in 

In [778]:
gen_ids

tensor([[   218,   2686,    113,     88,    256,  14170,    753,     48,     32,
           1075,    316,     20,     67,     11,  19282,  16740,     22,     32,
         122605,      5,      2,  73913,    129,    218,     25, 168217,    395,
            824,    102,  54862,    522,    247,  11076,  66092, 141249,  40165,
             64,   2494,    476,    218,     25,  46298,   5281,     27,    102,
           2774,      5,   7614,    129,    218]], device='cuda:0')

In [85]:
temp = 1
alpha = 1
eps = torch.finfo(torch.float32).eps
with torch.no_grad():
  c_lls = log_softmax(model(cond_prompt_t).logits[0][-1] / temp, dim=0)
  u_lls = log_softmax(model(uncond_prompt_t).logits[0][-1] / temp, dim=0)
  pmi = c_lls - u_lls
  pmi = pmi.long()
  pmi_dist = pmi - pmi.min() + eps
  pmi_dist = pmi_dist / pmi_dist.sum()
  pmi_lls = pmi_dist.log()
#   pmi.div_(pmi.sum())
#   apmi = alpha*pmi + (1-alpha)*torch.ones_like(pmi).div_(pmi.size(0))
# tokenizer.decode(torch.topk(apmi, 10).indices)
torch.topk(pmi_lls, 100, largest=False)

torch.return_types.topk(
values=tensor([-31.4399, -31.4399, -14.8043, -14.8043, -14.8043, -14.8043, -14.3989,
        -14.3989, -14.3989, -14.3989, -14.3989, -14.3989, -14.3989, -14.1112,
        -14.1112, -14.1112, -14.1112, -14.1112, -14.1112, -14.1112, -14.1112,
        -14.1112, -14.1112, -14.1112, -14.1112, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.

In [78]:
torch.Tensor([1.1160e-06]).log()

tensor([-13.7058])

In [1989]:
with torch.no_grad():
  print(torch.topk(log_softmax(model(cond_prompt_t).logits[0][-1] / temp), 10))

torch.return_types.topk(
values=tensor([-0.6235, -0.8735, -5.3750, -5.5000, -6.0000, -6.2500, -6.2500, -6.6250,
        -7.0000, -7.2500], device='cuda:0', dtype=torch.float16),
indices=tensor([   268, 139910,     46,    984,    345,  21992, 102595,     32,   2686,
           826], device='cuda:0'))


/tmp/ipykernel_52376/1101773222.py:2: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.topk(log_softmax(model(cond_prompt_t).logits[0][-1] / temp), 10))


In [775]:
prompt_ids = torch.LongTensor([tokenizer('translate into english, 中文： 谢谢你！').input_ids[:-1]]).to(model.device)

In [232]:
prompt_ids = torch.LongTensor([tokenize('中文： 谢谢你! english:')]).to(model.device)                          

In [233]:
prompt_ids.tolist()

[[6, 19773, 13, 6, 13305, 13305, 710, 35, 83763, 13]]

In [351]:
model.config

XGLMConfig {
  "_name_or_path": "facebook/xglm-2.9B",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "XGLMForCausalLM"
  ],
  "attention_dropout": 0.1,
  "attention_heads": 16,
  "bos_token_id": 0,
  "d_model": 2048,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 8192,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "xglm",
  "num_layers": 48,
  "pad_token_id": 1,
  "scale_embedding": true,
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 256008
}

In [275]:
with torch.no_grad():
  gen_ids = mt.sample(model, prompt_ids, 200)

TypeError: expected Tensor as element 0 in argument 0, but got list

In [309]:
tokenizer.decode(gen_ids[0].tolist())

'Leading researchers say that the test can detect cancer, tuberculosis, HIV and malaria in patients who live in low-income countries, where for example, the survival rate of breast cancer is half that of the richer countries.</s> Finnish.</s> Finnish : Hän sanoi, että hän sanoi, että hän sanoi, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että,'

In [231]:
print(tokenizer.decode(prompt_ids[0]))

中文: 谢谢你! | english:


In [14]:
inference_point['sentence']

'Stanfordin yliopiston lääketieteen laitoksen tutkijat ilmoittivat maanantaina uuden diagnostiikkatyökalun keksimisestä: solut tyypin mukaan lajitteleva pienenpieni tulostettava siru, joka voidaan valmistaa normaaleilla mustesuihkutulostimilla mahdollisesti noin yhden Yhdysvaltain sentin kappalehintaan.'

In [209]:
s_ids('')

[]

In [219]:
def tokenize(s):
  ids = []
  i = 0
  t = 0
  while t < len(s):
    c = s[t]
    if c == '\n':
      ids += s_ids(s[i:t]) + [2]
      i = t+1
      t = t+1
    else:
      t += 1
  ids += s_ids(s[i:])
  return ids

In [12]:
vocab = tokenizer.get_vocab()
tokens = list(sorted(tokenizer.get_vocab(), key=lambda k: vocab[k]))

In [121]:
tokens[:15]

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 ',',
 '.',
 '▁',
 's',
 '-',
 'a',
 '▁de',
 '▁a',
 'e',
 ':',
 'i']

In [305]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad

In [306]:
tokenizer.model_max_length

1000000000000000019884624838656

In [319]:
def make_demo_prompt(src_points, tgt_points, src_name, tgt_name, src, tgt, k, tokenizer):
    demos = []
    for s, t in zip(src_points, tgt_points):
        assert(s['id'] == t['id'])
        demo = f"{src_name} : {s['sentence']} {tgt_name} : {t['sentence']}"
        demos.append(demo)
    demo_prompt_ids = [2]
    for demo in demos:
      demo_prompt_ids.extend(mt.xglm_tokenize(demo, tokenizer))
      demo_prompt_ids.append(2)
    return demo_prompt_ids

In [323]:
src_points = random.sample(list(src['dev']), k=3)
tgt_points = [ tgt['dev'][point['id']-1] for point in src_points ]

In [334]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [330]:
print(mt.post_proc(tokenizer.decode(mt.make_demo_prompt(src_points, tgt_points, 'finnish', 'english', tokenizer))))


 finnish : Kiertoajelut ovat halvempia suurille ryhmille, joten jos liikut yksin tai vain yhden ystävän kanssa, koeta tutustua muihin ihmisiin ja muodostaa 4–6 hengen ryhmä parempaa yksilöhintaa varten. english : Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 finnish : Vaatimukset on tarkoitettu tekemään maiden välisestä muuttovirrasta järjestelmällisen. english : These requirements are designed to provide an organized migratory flow between both countries.
 finnish : Varo: pikkukaupunkien baarit eivät täälläpäin aina ole parhaita ajanviettopaikkoja vieraspaikkakuntalaisille. english : Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.



In [331]:
def make_uncond_demo_prompt(tgt_points, tokenizer):
    uncond_demo_prompt_ids = [2]
    for t in tgt_points:
      uncond_demo_prompt_ids.extend(mt.xglm_tokenize(t['sentence'], tokenizer))
      uncond_demo_prompt_ids.append(2)
    return uncond_demo_prompt_ids

In [333]:
print(mt.post_proc(tokenizer.decode(make_uncond_demo_prompt(tgt_points, tokenizer))))


 Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 These requirements are designed to provide an organized migratory flow between both countries.
 Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.

